In [1]:
import pandas as pd
import numpy as np
import re
dish = pd.read_csv("p02_dish_cleaned.csv.gz", compression='gzip')

In [2]:
def freq(col, printThis = True, printCount = -1):
    result = {}
    for row in col:
        if row in result: result[row] += 1
        else: result[row] = 1
    
    if printThis:
        sort = []
        for (k, v) in result.items(): sort.append((k,v))
        sort = sorted(sort, key = lambda x: -x[1])
        for el in sort:
            print(str(el[0]) + "\t" + str(el[1]))
            printCount = printCount - 1
            if printCount == 0: break
                
    return result

In [3]:
# This breaks up dish into tokens and collects them in a single placce.
# No need to rerun this unless you need to recrate tokens.csv
if None:
    results = {}
    for (index, row) in dish.iterrows():
        el = row['name'].split()
        for em in el:
            if em in results: results[em] += 1
            else: results[em] = 1

    results_df = pd.DataFrame({"token": list(results.keys()),
                            "frequency": list(results.values())})
    results_df.to_csv("tokens.csv", index = False)

In [4]:
# I imported tokens.csv into excel and manually labelled the first 50% of the tokens
tokens = pd.read_csv("tokens_cleaned.csv")
labels = {}
for (index, row) in tokens.iterrows():
    if pd.notna(row['action']):
        labels[row['token']] = (row['action'], row['value'])

In [5]:
# Each of the tokens comes with an action
#  del => "delete this word it is unimportant"
#  prep => "describes how the meal was prepared"
#  ing => "describes an ingredient used in the meal"
#  trans => "likely a foreign word that needs translation"
#  adj => "this is a generic adjective"
#  region => "this is a place that describes local cuisine"
#  quant => "something that describes the quantity of food items or size"

def applyTranslations(row):
    tok = row['name'].split()
    new_tok = []
    for t in tok:
        if t in labels and labels[t][0] == "trans": t = labels[t][1]
        new_tok.append(t)
    row['name_translated'] = " ".join(new_tok)
    return row

def applyManualLabels(row):
    tok = row['name_translated'].split()
    ing = []
    prep = []
    adj = []
    region = []
    quant = []
    brand = []
    unmatched_tokens = []
    for t in tok:
        if t in labels:
            a = labels[t][0]
            if a == "del": pass
            elif a == "ing": ing.append(t)
            elif a == "trans": pass
            elif a == "prep": prep.append(t)
            elif a == "adj": adj.append(t)
            elif a == "region": region.append(t)
            elif a == "quant": quant.append(t)
            elif a == "brand": brand.append(t)
            elif a == "?" or a == "extra":
                print(tok)
                pass
            else:
                print("Unkown action {0}".format(a))
                raise Exception()
        else: unmatched_tokens.append(t)
    row['unmatched_tokens'] = " ".join(unmatched_tokens)
    row['ingredients'] = " ".join(ing)
    row['preparation'] = " ".join(prep)
    row['adjectives'] = " ".join(adj)
    row['region'] = " ".join(region)
    row['quantity'] = " ".join(quant)
    row['brand'] = " ".join(brand)
    return row

In [6]:
dish2 = dish.apply(applyTranslations, axis = 1)

In [7]:
dish3 = dish2.apply(applyManualLabels, axis = 1)

['Brook', 'Trout,', 'Mountain', 'Style']
['Mohren', 'Suppe', 'Mit', 'Sago']
['Bratapfel', 'Mit', 'Schlagsahne']
['Sauternes,', 'Cruse', 'and', 'Fils', 'Freres']
['Sauternes,', 'J.', 'Calvet', 'and', 'Co.']
['White', 'Top,', 'Club', 'Special,', 'Lake', 'Kuka', 'Wine', 'Co.']
['J.', 'Calvet', 'and', 'Cie.']
['St.', 'Julien,', 'J.', 'Calvet', 'and', 'Co']
['Pontet', 'Canet,', 'J.', 'Calvet', 'and', 'Co.']
['Margaux,', 'Cruse', 'and', 'Fils', 'Freres']
['Chateau', 'Smith,', 'Haut', 'Lafite,', 'J.', 'Calvet', 'and', 'Co.']
['"', 'Cruse', 'and', 'Fils', 'Freres']
['Lake', 'Bass', 'Saute,', 'Meuniere']
['Lake', 'Trout']
['Fresh', 'Bratwurst', 'Mit', 'Sauerkraut,', 'Kartoffelbrei']
['Lynnhaven', 'Bay']
['Blue', 'Points', 'or', 'Lynnhaven', 'Bay', 'Oysters,', 'Fried', 'or', 'Broiled']
['Lachs', 'in', 'Sulze', 'Mit', 'Olguss']
['Kalbschroeser', 'Mit', 'Jungen', 'Erbsen']
['Roastbeef', 'Mit', 'Remoulade']
['Fisch', 'Mit', 'Sauce', 'Und', 'Potato']
['Roastbeef', 'Mit', 'Gemuse', 'Und', 'Beilage']


['Lake', 'Trout', 'Broiled', 'or', 'Fried']
['J.', 'Calvet', 'and', 'Co.,', 'St.', 'Julien']
['J.', 'Calvet', 'and', 'Co.,', 'Chateau', 'De', 'Montlys']
['J.', 'Calvet', 'and', 'Co.,', 'Chateau', 'Pontet', 'Canet,', '1887']
['J.', 'Calvet', 'and', 'Co.,', 'Chateau', 'Lagrange,', '1880']
['J.', 'Calvet', 'and', 'Co.,', 'Chateau', 'Larose,', '1877']
['J.', 'Calvet', 'and', 'Co.,', 'Chateau', 'Mouton', 'Rothschild,', '1880']
['Baked', 'Lake', 'Trout', 'Au', 'Wine', 'White']
['Cold', 'Brook', 'Trout']
['Lake', 'Trout,', 'Anchovy', 'Sauce']
['Diverse', 'Kase', 'Mit', 'Butter']
['Bordeaux', 'Vieux,', 'J.', 'Calvet']
['St.', 'Julien,', 'J.', 'Calvet']
['Chateau', 'Pontet', 'Canet,', 'J.', 'Calvet']
['Chateau', 'De', 'Montlys,', 'J.', 'Calvet']
['Chateau', 'Lagrange,', 'J.', 'Calvet']
['Graves,', 'J.', 'Calvet']
['Sauternes,', 'J.', 'Calvet']
['Apfelschnitte', 'Mit', 'Birnen']
['Sunny', 'Brook']
['Haut', 'Sauternes,', 'J.', 'Calvet', 'and', 'Co.']
['Chateau', 'Yquem,', 'J.', 'Calvet', 'and', '

['To', 'Order', '-', 'Fried', 'Brook', 'Trout,', 'with', 'Bacon']
['Chablis,', '1889', '(White),', 'J.', 'Calvet', 'and', 'Co.']
['Chambertin,', 'J.', 'Calvet', 'and', 'Co.']
['Macon', 'Vieux,', 'J.', 'Calvet', 'and', 'Co.']
['Graves', 'D.B.,', 'J.', 'Calvet', 'and', 'Co.']
['Chateau', 'Larose', '1877,', 'J.', 'Calvet', 'and', 'Co.']
['Chateau', 'Smith-Haut-Lafite,', 'J.', 'Calvet', 'and', 'Co.']
['Chateau', 'Pontet-Canet,', '1887,', 'J.', 'Calvet', 'and', 'Co.']
['La', 'Rose,', 'J.', 'Calvet', 'and', 'Co.']
['Montterand,', 'J.', 'Calvet', 'and', 'Co.']
['Haut', 'Sauternes,', 'Calvet', 'and', 'Co.']
['Graves,', 'Calvet', 'and', 'Co.']
['Kalbsbraten', 'Mit', 'Salas']
['Broiled', 'Lake', 'Erie', 'Trout,', 'Persilade']
['Gedampfte', 'Kallsbrust', 'Mit', 'Spinat']
['White', 'Wines', '-', 'Cruse', 'and', 'Fils', 'Freres']
['Geflugelleber', 'Mit', 'Truffeln']
['Rostbraten', 'Mit', 'Bratensaft']
['St.', 'Julien,', 'Journu', 'Freres']
['Bordeaux,', 'Brandenburg', 'Freres']
['Chateau', 'Beychev

['Eierkuchen', 'Mit', 'Parmesan-Kase']
['Schneeballe', 'Mit', 'Schlagrahm']
['Kraftbruhe', 'Mit', 'Eiergelee']
['Netzwurstchen', 'Von', 'Schweinefleisch', 'Mit', 'Spinat']
['Kraftbruhe', 'Mit', 'Sago']
['Rostbraten', 'Mit', 'Tafelpilzen']
['Zungen-Ragout', 'Mit', 'Oliven']
['Bouillon', 'Mit', 'Griesklossen']
['Gulasch', 'Ã¡', 'La', 'Minute']
['Kraftbruhe', 'Mit', 'Leberklossen.']
['Duck', 'A', 'Tete', 'Rouge']
['Frischer', 'Hummer', 'Mit', 'Remoladen', 'Sauce']
['Frischer', 'Hummer', 'Mit', 'Mayonaise']
['Junge', 'Erbsen', 'Mit', 'Carotten']
['Spritzkuchen', 'Mit', 'Orangensauce']
['Speck', 'Mit', 'Eier,']
['Omelette', 'Mit', 'Tomaten']
['Omelette', 'Mit', 'Parmesan']
['Omelette', 'Mit', 'Champignons,']
['Omelette', 'Mit', 'Spargel']
['Omelette', 'Mit', 'Feinen', 'Krautern,']
['Pfannfuchen', 'Mit', 'Marmelade']
['Pfannfuchen', 'Mit', 'Kronsbeeren']
['Grill.', 'Lachs', 'Mit', 'Jungen', 'Erbsen']
['Rumpsteak', 'Mit', 'Krauterbutter']
['Kalbsleber', 'Mit', 'Speck', 'Und', 'Zwiebeln']
['Ge

['G.', 'H.', 'Mumm', 'Cordon', 'Rouge', '(Champagne)']
['Sorbet,', 'Cordon', 'Rouge']
['Sauterne', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Haut', 'Sauterne', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Johnston', 'and', 'Sons', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Count', 'Saluce', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Latour', 'Blanche,', 'Sweet', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Chablis', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Yquem', 'Haut,', '1847', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Chateau', 'Yquem', 'Tete', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Chateau', 'Yquem', 'Cream', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['St.', 'Julien,', 'Selected', 'By', 'P.', 'Stevens', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['St.', 'Julien,', 'First', 'Quality,', 'Selected', 'By', 'P.', 'Stevens', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['La', 'Rose,', 'Selected', 'By', 'P.', 'Stevens', 'From', 'Cruse', 'and', 'Fils', 'Freres']
['Chateau', 'Pomys

...

['Broiled', 'Speckled', 'Brook', 'Trout', 'Served', 'with', 'Two', 'Vegetables', 'and', 'Chef', 'Salad', 'Bowl']
['Broiled', 'Speckled', 'Brook', 'Trout', 'Served', 'with', 'Two', 'Vegetables', 'and', 'Cole', 'Slaw']
['Broiled', 'Speckled', 'Brook', 'Trout', 'Served', 'with', 'Two', 'Vegetables', 'and', 'Fresh', 'Chopped', 'Salad']
['Broiled', 'Speckled', 'Brook', 'Trout', 'Served', 'with', 'Two', 'Vegetables', 'and', 'Lettuce', 'and', 'Tomato']
['Broiled', 'Speckled', 'Brook', 'Trout', 'Served', 'with', 'Two', 'Vegetables', 'and', 'Chef', 'Salad', 'Bowl', 'or', 'Cole', 'Slaw', 'and', 'Fresh', 'Garlic', 'or', 'Fresh', 'Onions']
["Hudson'S", 'Bay', 'Scotch']
['Hudson', 'Bay', '15', 'Yrs.', 'Old', 'Scotch']
['Filet', 'of', 'Sea', 'Bass', 'Sautã©', 'with', 'Grapes', 'and', 'Almonds,', 'Vã©Ronique,', 'Potatoes', 'Minute']
['Minute', 'Steak', 'Sautã©', 'with', 'Potatoes', "O'Brien,", 'Fresh', 'String', 'Beans']
['Latour', 'Blanche', 'Brandenburg', 'Freres', 'Bordeaux', 'White', 'Wine']
['Ka

['Broiled', 'Brook', 'Trout', 'Meuniã¨Re']
['Klare', 'Schildkrã¶Tensuppe', '"Lacroix"', 'Mit', 'Altem', 'Sherry']
['Mild', 'Gerã¤Uchertes', 'Forellenfilet', 'Mit', 'Oberskren']
['Gefã¼Lltes', 'Melonenschiffchen', 'Mit', 'Shrimps', 'Und', 'Kã¤Sewã¼Rfeln']
['Hã¼Hnerkraftbrã¼He', 'Mit', 'Hã¼Hnerfleisch', 'Und', 'Spargelspitzen']
['Gefã¼Llte', 'Grapefruit', '"Washington"', 'Mit', 'Shrimps', 'Und', 'Pikater', 'Cocktailsauce']
['Westfã¤Ler', 'Oder', 'Bã¼Ndner', 'Fleisch', 'Mit', 'Kren', 'Und', 'Essiggurkerl']
['Langustenschwanz', 'Mit', 'Sauce', 'Mornay', 'Ã¼Berbacken', '(Ca.', '100', 'G)', 'Patnareis']
['Froschschenkel', 'Mit', 'Zwiebelpã¼Ree', 'Ã¼Berbacken,', 'Patna', 'Reis']
['6', 'Stã¼Ck', 'Franzã¶Sische', 'Weinbergschnecken', 'Mit', 'Duftender', 'Knoblauchbutter', 'Und', 'Haselnã¼Ssen', 'Gefã¼Llt']
['Schinkenrolle', '"Black', 'Jack"', 'in', 'Schinkenscheibe', 'Eingewickelte', 'Banane,', 'Mit', 'Emmentaler', 'Kã¤Se', 'Ã¼Berbacken']
['Vintschgerl', 'Mit', 'Mildgerã¤Ucherten', 'Bauernspeck

['Champagne:', 'G', 'H', 'Mumm', 'Cordon', 'Rouge']
['Gegrillter', 'Magerer', 'Katenschinken', 'Auf', 'Toast', 'Mit', 'Rindermark', 'Und', 'Parmesan', 'Gratiniert']
['4', 'Stuck', 'Schwedenhã¤Ppchen', 'Mit', 'Feinem', 'Fleischsalat']
['Confit', 'De', 'Lã©Gumes', 'Au', 'Chou', 'Rouge', 'Et', 'Liver', 'Gras', 'De', 'Duck', 'Poã«Lã©']
['Kã¤Lberfã¼Ã\x9fE', 'Abgebrã¤Unt', 'Mit', 'Gemischtem', 'Salad']
['1/2', 'Helgolaender', 'Hummer', 'Mit', 'Toast', 'Und', 'Mayonnaise']
['Omelette', 'Mit', 'Ham', 'Oder', 'Mushrooms', 'Und', 'Grã¼Nem', 'Salad', '8,', '60']
['Omelette', 'Mit', 'Ham', 'Oder', 'Mushrooms', 'Oder', 'Kã¤Se', 'Und', 'Grã¼Nem', 'Salad', '8,60']
['4', 'Stã¼Ck', 'Schwedenhã¤Ppchen', 'Mit', 'Feinem', 'Fleischsalat']
['Kã¶Stlicher', 'Salad', 'Von', 'Fr.', 'Pfirsichen', 'Mit', 'Cointreau', 'Am', 'Tisch', 'Zubereitet']
['Kasseler', 'Rippenspeer', 'Auf', 'Ananaskraut', 'Mit', 'Kartoffelpã¼Rã©E']
['Schwã¤Bische', 'Flã¤Dlesuppe,', 'Schweinebraten', 'Glaciert', 'Mit', 'Kartoffelklã¶Ã\x9fEn'

['Forelle', 'Blau', 'Mit', 'Zerlassener', 'Butter,', 'Salzkartoffeln', 'Und', 'Salad']
['Warme', 'Brã¼Hwoosch', 'Mit', 'Adappelschlaat']
['Ein', 'Brotwoosch', 'Mit', 'Zaus', 'on', 'Kappes-Puree']
['Schweinekotelett,', 'Kalt', 'Mit', 'Kartoffelsalat']
['Schweinebraten,', 'Kalt', 'Mit', 'Remouladen-Sauce']
['Krustchen', 'Rindfleisch', 'Mit', 'Meerrettich']
['Rundstã¼Ck', 'Warm', 'Mit', 'Schweinebraten']
['Angel-Schellfisch', 'Mit', 'Senfbutter', 'Und', 'Kartoffeln.']
['Breast', 'of', 'Duckling', 'Sautã©,', 'Minute']
['Caviar', '(Deutscher)', 'Op', ',,Russisch', 'Ei"', '4', 'Halve', 'Egg', 'Mit', 'Schlaat', 'Dobei']
['Leckere', 'Reppches', 'Mit', 'Adappelschlaat']
['Kalbssteak', 'Mit', 'Spargelspitzen,', 'Zerlassene', 'Butter,', 'Potato']
['1', 'Paar', 'Original', 'Frankfurter', 'Wã¼Rstchen', 'Mit', 'Mayonnaise-Salat']
['Broiled', 'Steak', 'Minute']
['Brusseler', 'Poularde', 'Mit', 'Kompott']
['Special', 'Steak:', 'Minute', 'Steak,', 'French', 'Fried', 'Potato,', 'Vegetable']
['Cow', 'Bay

['J', 'Calvet', 'and', "Co'S", 'Sauternes']
['Perdreau', 'Rouge']
['Karfiol', 'Mit', 'Sce', 'Polonaise']
['Spargel', 'Mit', 'Butter']
['Engl.', 'Speck', 'Gebraten,', 'Mit', 'Ei']
['Grilled', 'Brook', 'Trout', 'Over', 'A', 'Warm', 'Spinach,', 'Walnut', 'and', 'Lentil', 'Salad']
['Fresh', 'Brook', 'Trout']
['Schildkrã¶Tensuppe', '"Lady', 'Curzon"', 'Mit', 'Currysahne', 'Ã¼Berbacken']
['Hausmacher', 'Sã¼Lze', 'Mit', 'Kartoffel', 'Salad']
['Hã¼Hnerbrã¼He', 'Mit', 'Einlage', 'Oder', 'Ei']
['Linsen', 'Suppe', 'Mit', 'Wã¼Rstchen']
['Schweinsrã¼Cken', 'Mit', 'Carotten', 'Und', 'Erbsen']
['Sã¼Lzkotelette', 'Garniert', 'Mit', 'Kopfsalat', 'Und', 'Bratkartoffeln']
['Nordsee-Steinbutt', 'Gebraten', 'Mit', 'Gemischtem', 'Und', 'Mayonnaisensalat']
['Kalbsleber', 'Mit', 'Feinen', 'Krã¤Utern']
['Doppelte', 'Kraftbrã¼He', 'Mit', 'Mark']
['Bay', 'Scallops', 'Broiled']
['Gartenkrã¤Utersalat', 'Mit', 'Sherryessig-Vinaigrette', 'Gebratenem', 'Spargel', 'Und', 'Gerã¶Steten', 'Pinienkernen']
['Provencialisch

In [8]:
x = freq(dish3['ingredients'], printCount = 20)

	162203
Chicken	6218
Wine	3144
Potatoes	2976
Beef	2726
Butter	2378
Ham	2165
Lamb	2164
Cheese	2127
Fries	2074
Veal	2048
Lobster	2026
Ice	1864
Egg	1704
Champagne	1690
Potato	1686
Eggs	1593
Tomato	1411
Fruit	1386
Mushrooms	1386


In [9]:
x = freq(dish3['region'], printCount = 20)

	350463
French	8439
Chateau	3212
English	1680
American	1213
Virginia	1000
Boston	965
California	934
Spanish	909
Italian	843
Parisienne	824
Bordelaise	804
Maryland	777
Madeira	698
Bordeaux	687
York	675
German	667
Rhine	649
Newburg	648
Russian	644


In [11]:
x = freq(dish3['adjectives'], printCount = 20)

	312796
White	3649
Extra	2749
New	2462
Spring	2361
Red	2162
Sweet	1984
House	1877
Old	1768
Prime	1460
Vanilla	1337
Breast	1317
Supreme	1308
Plain	1289
Choice	1267
Assorted	1246
Les	1225
Club	1204
Chops	1196
Sea	1142


In [12]:
x = freq(dish3['preparation'], printCount = 20)

	209676
Cream	7540
Salad	7434
Sauce	6546
Broiled	4352
Fried	4255
Fresh	3277
Soup	2909
Roast	2692
Filet	2575
Dry	2378
Cold	2173
Consomme	2077
Baked	2011
Imported	1769
Saute	1593
Jelly	1568
Boiled	1553
Glazed	1491
Brut	1482


In [13]:
x = freq(dish3['quantity'], printCount = 20)

	351124
Half	3046
Cup	2030
2	1695
Two	1489
Small	1392
1/2	1345
Grand	1088
1	1063
Glass	1050
per	805
Long	794
For	784
Jumbo	772
Large	770
Bowl	729
Little	719
Petits	675
Double	654
Bottle	653


In [15]:
x = freq(dish3['unmatched_tokens'], printCount = 20)

	56682
Pot	606
(2)	392
Country	299
Maitre	270
Tips	265
Home	224
(1)	205
W.	191
Herbes	171
Saddle	165
Hash,	159
Mackerel	157
Imperial	153
Shoulder	149
Suey	148
Tenderloin,	147
Graham	145
Dinner	144
Jersey	140


In [16]:
dish3.head()

,id,name,name_cluster,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price,name_translated,unmatched_tokens,ingredients,preparation,adjectives,region,quantity,brand
0,5,St. Emilion,St. Emilion,66,68,1881.0,1981.0,0.0,18.0,St. Emilion,St. Emilion,,,,,,
1,10,"Cream of New Asparagus, Croutons","Cream of new asparagus, croutons",2,2,1900.0,1900.0,0.0,0.0,"Cream of New Asparagus, Croutons",,"Asparagus, Croutons",Cream,New,,,
2,14,Fresh Lobsters in every Style,Fresh lobsters in every style,4,4,1899.0,1900.0,0.0,0.0,Fresh Lobsters in every Style,every,Lobsters,Fresh,,,,
3,19,India Chutney,India chutney,16,16,1865.0,1901.0,0.1,0.2,India Chutney,India Chutney,,,,,,
4,22,Pate De Foies-Gras,Pate de foies-gras,9,9,1898.0,1901.0,1.0,1.0,Pate De Foies-Gras,Foies-Gras,,Pate,,,,


In [17]:
dish3.to_csv("p03_dish_cleaned.csv.gz", compression='gzip', index = False)